In [1]:
%pwd

u'/home/koldo/Dropbox/Proyectos/Data_Science/Second-hand_vehicles_price_estimator'

In [5]:
import pandas as pd
df_motos_raw = pd.read_csv('scraped_data/motos_raw_data.csv', sep=';')
df_motos_raw.shape

(26511, 10)

In [6]:
df_motos_raw.head(5)

,city,brand,model,type,cc,color,km,year,price,url
0,Madrid,BMW,R 1200 GS,Trail,1170.0,ROJO,57400.0,2004,7490,http://motos.net/ocasion/bmw/r_1200_gs/2004-en...
1,Navarra,KYMCO,Agility 125,Scooter 125cc,125.0,NaN,7000.0,2014,1600,http://motos.net/ocasion/kymco/agility_125/201...
2,Navarra,KTM,390 Duke,Naked,375.0,NaN,9400.0,2014,4700,http://motos.net/ocasion/ktm/390_duke/2014-en-...
3,Navarra,HONDA,CBR 125R,Sport,125.0,NaN,2100.0,2014,2800,http://motos.net/ocasion/honda/cbr_125r/2014-e...
4,Granada,SUZUKI,BURGMAN 250,Scooters +125cc,250.0,NaN,57000.0,2006,1300,http://motos.net/ocasion/suzuki/burgman_250/20...


Data cleaning

In [7]:
# Convert the dataframe to lower case
df_motos_raw = df_motos_raw.apply(lambda x: x.astype(str).str.lower())

In [8]:
#Join the words in the 'model' and 'type' fields with an underscore
df_motos_raw['model'] = df_motos_raw['model'].str.replace(' ', '_')
df_motos_raw['type'] = df_motos_raw['type'].str.replace(' ', '_')

In [9]:
# Looking for duplicates
df_motos_raw['is_duplicated'] = df_motos_raw.duplicated()
duplicates = df_motos_raw['is_duplicated'].sum()
print '%d duplicates' %duplicates

93 duplicates


In [10]:
# Removing duplicates and delete 'is_duplicated' column
df_motos_raw = df_motos_raw.loc[df_motos_raw['is_duplicated']==False]
df_motos_raw = df_motos_raw.drop('is_duplicated', 1)
df_motos_raw.shape

(26418, 10)

In [11]:
# Lets investigate column by column the NaNs we have in the dataframe
for column in df_motos_raw.columns:
    n_nan = df_motos_raw[column]=='nan'
    print column + " %d -- > %f" %(n_nan.sum(), (n_nan.sum()*1.0)/df_motos_raw.shape[0]*100)

city 0 -- > 0.000000
brand 0 -- > 0.000000
model 0 -- > 0.000000
type 0 -- > 0.000000
cc 95 -- > 0.359603
color 7212 -- > 27.299568
km 1631 -- > 6.173821
year 0 -- > 0.000000
price 0 -- > 0.000000
url 0 -- > 0.000000


In [12]:
# Lets investigate the unique values we have in the columns
for column in ['city', 'brand', 'model', 'type', 'color', 'year']:
    column_uv = df_motos_raw[column].unique()
    print column + " --> " + "%d unique values" %len(sorted(column_uv))

city --> 52 unique values
brand --> 148 unique values
model --> 3011 unique values
type --> 15 unique values
color --> 1923 unique values
year --> 48 unique values


Merge some data

In [13]:
#In order to calculate our first metric, we will use the following columns:
# "lon" and "lat" : These are the longitude and latitude of the corresponding city. We will add them later.
# "brand", "model", "type", "year"

In [14]:
# Calculating the longitude and latitude of the cities
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [15]:
cities = df_motos_raw['city'].unique()
locations_rows = []
for city in cities:
    location = geolocator.geocode([city], timeout = 15)
    locations_rows.append([city, location.latitude, location.longitude])
#Save into a csv
df_locations = pd.DataFrame(locations_rows, columns = ['city', 'lat', 'lon'])
df_locations.to_csv('auxiliary_data/locations_coords.csv', index = False)

In [16]:
df_locations.head(3)

,city,lat,lon
0,madrid,40.416705,-3.703582
1,navarra,42.612549,-1.830791
2,granada,37.183020,-3.602192


In [17]:
# Merge df_locations with df_motos_raw
df_motos_raw_coord = pd.merge(df_motos_raw, df_locations, on = 'city')
#Save into a csv
df_motos_raw_coord.to_csv('df_motos_raw_coord.csv', index = False)

In [18]:
# We have created two csv files with a rank for the motos brands and types
# called rank_motos_brands.csv and rank_motos_types.csv

# With the first one, rank_motos_brands.csv, we are gint to create another  column, with a score to the corresponding brand
df_rank_moto_brand = pd.read_csv('scraped_data/rank_moto_brands.csv', sep=';')
df_motos_raw_coord_brand = pd.merge(df_motos_raw_coord, df_rank_moto_brand, on = 'brand', how = 'left')
#If the brand does not exist, the rank value will be zero
#!!OJO, AÑADIR MARCAS DE MOTOS QUE FALTAN ANTES DE PONER UN CERO!!!!!!!!!!!!!!!!!
df_motos_raw_coord_brand.brand_score.fillna(0, inplace=True)
# Save into a csv
df_motos_raw_coord_brand.to_csv('df_motos_coord_brand.csv', index = False)

# With the second one, rank_moto_types.csv, we are going to create another column, with a score to the corresponding type
df_rank_moto_type = pd.read_csv('auxiliary_data/rank_moto_types.csv', sep=';')
df_motos_raw_coord_brand_types = pd.merge(df_motos_raw_coord_brand, df_rank_moto_type, on = 'type', how = 'left')
#Save into a csv
df_motos_raw_coord_brand_types.to_csv('df_motos_raw_coord_brand_type.csv', index = False)

In [19]:
df_motos_raw_coord_brand_types.head(3)

,city,brand,model,type,cc,color,km,year,price,url,lat,lon,brand_score,type_score
0,madrid,bmw,r_1200_gs,trail,1170.0,rojo,57400.0,2004,7490,http://motos.net/ocasion/bmw/r_1200_gs/2004-en...,40.416705,-3.703582,356.0,14
1,madrid,ducati,monster_696+,naked,696.0,blanco,18166.0,2011,5999,http://motos.net/ocasion/ducati/monster_696/20...,40.416705,-3.703582,473.0,15
2,madrid,kymco,super_dink_300i_abs,scooters_+125cc,299.0,blanco,9644.0,2011,3299,http://motos.net/ocasion/kymco/super_dink_300i...,40.416705,-3.703582,37.0,10


In [20]:
df_motos_raw_coord_brand_types[df_motos_raw_coord_brand_types.brand_score==0].brand.unique()

array(['dorton', 'mv agusta', 'quadro', 'indian', 'hanway', 'lml',
       'jonway', 'qingqi', 'royal enfield', 'riya', 'monkey bike', 'znen',
       'motor hispania', 'fantic', 'fb mondial', 'imr', 'hudson boss',
       'rav', 'jin lun', 'oset', 'orcal', 'big dog', 'brp', 'montesa',
       'ural', 'kinroad', 'mtr', 'zero', 'mash', 'mxonda', 'ksr moto',
       'puch', 'jianshe', 'innocenti', 'arctic cat', 'zongshen',
       'ridley motorcycles', 'american ironhorse', 'young rider', 'orion',
       'mobilette', 'tm', 'torrot', 'vespino', 'moto morini',
       'apollo orion', 'rebel', 'megelli', 'cooltra', 'lem', 'ossa',
       'zero motorcycles', 'vertigo', 'govecs', 'can-am', 'kangxin',
       'zms motors', 'hm', 'dh haotian', 'alpina renania', 'gowinn',
       'xmotos', 'lambretta', 'brammo', 'scomadi', 'aeon', 'lifan', 'ycf',
       'elmoto', 'renault', 'ajp', 'wildlander', 'sumo', 'mecatecno',
       'wottan', 'lemev', 'e-max', 'samada', 'btm', 'vmoto', 'tbq', 'mtm',
       'lanvertt

In [21]:
# OK!, so we have a first version of the data we will use to recommend vehicles
# df_motos_raw_coord_brand_types
# Lets try to calculate the metric only with some fields. We are going to add these distances:
# cities distance, brand_distance, type_distance, year_distance
# We need to create some functions:

def cities_distance(city_lat, city_lon, user_lat, user_lon):
    """    
    :param city_lat: the value in the dataset's lat column to the corresponding city
    :param city_lon: the value in the dataset's lon column to the corresponding city
    :param user_lat: The corresponding lat value in the location dataset of the city selected by the user
    :param user_lon: The corresponding lon value in the location dataset of the city selected by the user
    
    :return: The value in kilometers of the distance between the two cities.
    
    Usage of the Vicenty distance
    """
    
    from geopy.distance import vincenty
    
    column_city = (city_lat, city_lon)
    user_city = (user_lat, user_lon)
    
    return (vincenty(column_city, user_city).km)

def distance_abs_value(a_value, b_value):
    return abs(a_value - b_value)

def w_s(city_row, brand_row, type_row, year_row):
    import numpy as np
    weigth = 100
    brand_weight = 40
    type_weight = 40
    year_weight = 10
    city_weight = 10
    
    params = np.array([city_row, brand_row, type_row, year_row])
    weights = np.array([city_weight, brand_weight, type_weight, year_weight])
    
    num = sum(params * weights) * 1.0
    return num/weigth

In [26]:
#Example of request of a user
user_request = ('madrid', 'bmw', 'custom', 2000)

# We need to calculate some variables:
user_lat = float(df_locations[df_locations['city'] == user_request[0]].lat)
user_lon = float(df_locations[df_locations['city'] == user_request[0]].lon)
user_brand = int(df_rank_moto_brand[df_rank_moto_brand['brand'] == user_request[1]].brand_score)
user_type = int(df_rank_moto_type[df_rank_moto_type['type'] == user_request[2]].type_score)
user_year = user_request[3]

user_vars = [user_brand, user_type, user_year]
score_columns = ['brand_score', 'type_score', 'year']

In [27]:
df_motos_raw_coord_brand_types['city_metric'] = df_motos_raw_coord_brand_types.apply(lambda row: cities_distance(row['lat'], row['lon'], user_lat, user_lon), axis=1)

for i, element in enumerate(['brand', 'type', 'year']):
    new_column = element + '_metric'
    print new_column, score_columns[i], user_vars[i]
    df_motos_raw_coord_brand_types[new_column] = df_motos_raw_coord_brand_types.apply(lambda row: distance_abs_value(int(row[score_columns[i]]), user_vars[i]), axis=1)
    
#!!! Revisar ranking de motos, falta alguna, por eso aparenden NAN en la columna brand_score al hacer el merge

df_motos_raw_coord_brand_types['total_metric_pond'] = df_motos_raw_coord_brand_types.apply(lambda row: w_s(row['city_metric'], row['brand_metric'], row['type_metric'], row['year_metric']), axis = 1)

brand_metric brand_score 356
type_metric type_score 13
year_metric year 2000


In [28]:
results = df_motos_raw_coord_brand_types.sort_values(by = ['total_metric_pond'], ascending=True)

In [29]:
results.head(5)

,city,brand,model,type,cc,color,km,year,price,url,lat,lon,brand_score,type_score,city_metric,brand_metric,type_metric,year_metric,total_metric_pond
4920,madrid,bmw,r_1200_c_classic,custom,1170.0,cobre,33000.0,2000,9800,http://motos.net/ocasion/bmw/r_1200_c_classic/...,40.416705,-3.703582,356.0,13,0.0,0,0,0,0.0
1921,madrid,bmw,f_650,trail,652.0,rojo,40848.0,2000,3000,http://motos.net/ocasion/bmw/f_650/2000-en-mad...,40.416705,-3.703582,356.0,14,0.0,0,1,0,0.4
567,madrid,bmw,r_1200_c_montauk,custom,1200.0,nan,148000.0,2004,7300,http://motos.net/ocasion/bmw/r_1200_c_montauk/...,40.416705,-3.703582,356.0,13,0.0,0,0,4,0.4
2011,madrid,bmw,f_650_gs_dakar,trail,652.0,blanca,75000.0,2000,2990,http://motos.net/ocasion/bmw/f_650_gs_dakar/20...,40.416705,-3.703582,356.0,14,0.0,0,1,0,0.4
2363,madrid,bmw,r_1200_c,custom,1170.0,negro,16000.0,2004,9500,http://motos.net/ocasion/bmw/r_1200_c/2004-en-...,40.416705,-3.703582,356.0,13,0.0,0,0,4,0.4
